In [ ]:
!pip install diffusers transformers accelerate datasets huggingface_hub

In [ ]:
from huggingface_hub import login
from datasets import load_dataset

dataset = load_dataset("kevinbenoy/anime_random_images", split = "train")


README.md:   0%|          | 0.00/286 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/523M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/562M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
len(dataset)

20000

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())


True
1


## Prepare Dataset for Training

### Subtask:
The loaded dataset will be preprocessed for training the diffusion model. This involves resizing images to a standard dimension, normalizing pixel values, and converting them into PyTorch tensors. I will also remind you that the current dataset size of 10 images is extremely small for training a meaningful diffusion model.


**Reasoning**:
Now that the `preprocess` function is defined, I will apply it to the `dataset` using the `.with_transform()` method to prepare the images for model training. I will also add a markdown block to emphasize the point that the dataset size of 10 images is too small.



In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

def transform(examples):
    examples["pixel_values"] = [preprocess(img.convert("RGB")) for img in examples["image"]]
    return examples

dataset = dataset.with_transform(transform)


**Reasoning**:
The previous code failed because the `dataset` variable was not defined in the cell where `set_transform` was called, even though it was loaded in a preceding cell. To ensure `dataset` is available, I will re-load and select the dataset within the same code block before applying the transformations.



In [ ]:
from diffusers import UNet2DModel, DDPMScheduler

# Initialize the UNet2DModel
model = UNet2DModel(
    sample_size=64,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(64, 128, 128, 256),
)

print("UNet2DModel initialized successfully.")

# Initialize the DDPMScheduler
scheduler = DDPMScheduler(num_train_timesteps=1000)
print("DDPMScheduler initialized successfully.")

UNet2DModel initialized successfully.
DDPMScheduler initialized successfully.


## Configure and Train Diffusion Model

### Subtask:
Configure training parameters and execute the training loop for the diffusion model. This will involve setting up the optimizer, learning rate, and a limited number of training steps. Acknowledge the small dataset size and potential lack of GPU.


Given the extremely small dataset size (10 images) and the current lack of GPU, the training process will be configured with a limited number of epochs and will run on the CPU. It is crucial to understand that the resulting model will likely be of very low quality and will not represent a meaningfully trained diffusion model. This step primarily serves as a demonstration of the training loop rather than an attempt to produce a functional model.

**Reasoning**:
I will define the training parameters, create a DataLoader, initialize the AdamW optimizer, and implement the training loop as specified, moving the model and data to the CPU, and calculating the loss for each epoch.



In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. Define training parameters
num_epochs = 50
batch_size = 32  # Adjust if 10 images / batch_size is not an integer. For 10 images, batch_size could be 2, 5, or 10.
learning_rate = 1e-4

# 3. Create a DataLoader from the preprocessed dataset
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 4. Initialize the AdamW optimizer for the model
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 5. Move the model to the 'cpu' device
model.to(device)

# 6. Implement the training loop
model.train() # Set model to training mode

print(f"Starting training for {num_epochs} epochs on {device}...")

for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in train_dataloader:
        # i. Move the input images to the 'cpu' device
        clean_images = batch["pixel_values"].to(device)


        # ii. Generate random noise with the same shape as the images
        noise = torch.randn(clean_images.shape).to(device)

        # iii. Sample random timesteps between 0 and scheduler.config.num_train_timesteps
        timesteps = torch.randint(
            0, scheduler.config.num_train_timesteps, (clean_images.shape[0],)
        ).to(device).long()

        # iv. Add noise to the original images using scheduler.add_noise()
        noisy_images = scheduler.add_noise(clean_images, noise, timesteps)

        # v. Predict the noise using the model
        noise_pred = model(noisy_images, timesteps).sample

        # vi. Calculate the Mean Squared Error (MSE) loss
        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        # vii. Perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # d. Print the loss for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_dataloader):.4f}")

print("Training complete.")

Using device: cuda
Starting training for 50 epochs on cuda...


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.JpegImagePlugin.JpegImageFile'>

## Save Trained Model to Hugging Face

### Subtask:
Authenticate your Hugging Face account and save the trained diffusion model to a new repository on your Hugging Face profile.


**Reasoning**:
I will import `create_repo` from `huggingface_hub`, call `login()` to authenticate, define a `repo_id`, create the repository using `create_repo`, and then save the `model` and `scheduler` to that repository.



In [ ]:
from huggingface_hub import login, create_repo

# Log in to Hugging Face (you will be prompted to enter your token)
login()

# Define your repository ID
# Replace 'your-username' with your actual Hugging Face username
repo_id = "Tomisin05/unconditional-anime-diffusion-model"

# Create the repository on Hugging Face
create_repo(repo_id, exist_ok=True, repo_type="model")
print(f"Hugging Face repository '{repo_id}' created or already exists.")

# Save the trained model and scheduler to the repository
model.push_to_hub(repo_id)
scheduler.push_to_hub(repo_id)

print("Model and scheduler saved to Hugging Face repository.")

In [ ]:
from diffusers import DDPMPipeline
import torch

# Create a DDPMPipeline object from the trained model and scheduler
pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

pipeline.to("cuda")

# Define the total number of images to generate and a suitable batch_size
num_images_to_generate = 10
batch_size = 8  # Adjust based on memory availability

# Initialize an empty list to store the generated images
test_images = []

print(f"Starting generation of {num_images_to_generate} images...")

# Iterate to generate images in batches
for i in range(0, num_images_to_generate, batch_size):
    # Generate a batch of images, ensuring the pipeline is moved to the 'cpu' device
    # The previous cell trained on CPU, so we continue to generate on CPU.
    with torch.no_grad(): # Disable gradient calculations for inference
        images_batch = pipeline(batch_size=batch_size, output_type="pil").images

    # Extend the test_images list with the newly generated batch of images
    test_images.extend(images_batch)

    if (i + batch_size) % 10 == 0 or (i + batch_size) >= num_images_to_generate:
        print(f"Generated {len(test_images)} / {num_images_to_generate} images...")

# Print a confirmation message
print(f"Successfully generated {len(test_images)} images.")


In [ ]:

import matplotlib.pyplot as plt

# Display a few of the generated images
print(f"Displaying {min(5, len(test_images))} sample generated images:")

plt.figure(figsize=(10, 2))
for i, image in enumerate(test_images[:5]):
    plt.subplot(1, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
plt.show()

## Generate 20,000 Images

### Subtask:
Using the trained diffusion model, generate 20,000 new images. The generation process will be configured to ensure diversity and quality as much as possible, given the model's training limitations.


**Reasoning**:
I will import necessary libraries, instantiate the DDPM pipeline from the previously trained model and scheduler, then iterate to generate 20,000 images in batches, collecting them in a list.



In [ ]:
import os

# Create a DDPMPipeline object from the trained model and scheduler
pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

# Define the total number of images to generate and a suitable batch_size
num_images_to_generate = 21000
batch_size = 8  # Adjust based on memory availability

os.makedirs("generated_images", exist_ok=True)


print(f"Starting generation of {num_images_to_generate} images...")

count = 0
for i in range(0, num_images_to_generate, batch_size):
    with torch.no_grad():
        images = pipeline(batch_size=batch_size).images
    for img in images:
        img.save(f"generated_images/{count}.png")
        count += 1


# Print a confirmation message
print(f"Successfully generated {count} images.")

## Display Sample Generated Images

### Subtask:
Display a few of the recently generated images from the `generated_images` list to visualize the current output of the diffusion model.


In [ ]:
import matplotlib.pyplot as plt

import glob
from PIL import Image

sample_files = sorted(glob.glob("generated_images/*.png"))[:5]
images = [Image.open(f) for f in sample_files]

# Display a few of the generated images
print(f"Displaying {min(5, len(images))} sample generated images:")

plt.figure(figsize=(10, 2))
for i, image in enumerate(images):
    plt.subplot(1, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
plt.show()

## Create and Push Generated Image Dataset to Hugging Face

### Subtask:
The previously generated images will be compiled into a new Hugging Face dataset. This dataset will then be uploaded to a new repository on your Hugging Face profile, making it accessible for future use.


**Reasoning**:
I will import the `Dataset` class, create a dictionary from the `generated_images`, instantiate a Hugging Face Dataset, define a new repository ID, and then push the dataset to the Hugging Face Hub.



In [ ]:
import datasets
from datasets import Dataset, Image


ds = Dataset.from_dict(
    {"image": [f"generated_images/{i}.png" for i in range(count)]}
).cast_column("image", Image())

ds.push_to_hub("Tomisin05/generated-anime-images", commit_message="Upload 21k images", force_push=True)


### Generate a sample image

In [8]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 86.5 MB/s eta 0:00:00


In [9]:
!uv pip install --upgrade transformers diffusers torch

Using Python 3.12.12 environment at: /usr
Resolved 45 packages in 278ms
Prepared 11 packages in 36.32s
Uninstalled 10 packages in 1.08s
Installed 11 packages in 345ms
 - fsspec==2025.3.0
 + fsspec==2025.10.0
 - numpy==2.0.2
 + numpy==2.3.5
 + nvidia-cublas-cu12==12.8.4.1
 - nvidia-cufft-cu12==11.3.0.4
 + nvidia-cufft-cu12==11.3.3.83
 - nvidia-cusolver-cu12==11.7.1.2
 + nvidia-cusolver-cu12==11.7.3.90
 - nvidia-cusparse-cu12==12.5.4.2
 + nvidia-cusparse-cu12==12.5.8.93
 - pillow==11.3.0
 + pillow==12.0.0
 - requests==2.32.4
 + requests==2.32.5
 - setuptools==75.2.0
 + setuptools==80.9.0
 - torch==2.9.0+cu126
 + torch==2.9.1
 - transformers==4.57.2
 + transformers==4.57.3


In [11]:
from diffusers import DDPMPipeline
repo = "Tomisin05/unconditional-anime-diffusion-model"
model = DDPMPipeline.from_pretrained(repo)

AttributeError: module 'sympy' has no attribute 'core'